# Data Cleaning and Basic Encoding

In [39]:
import pandas as pd
import numpy as np
from clean_data import clean_data
from cat_encodings import count_encoding, count_rank_encoding, Holdout_target_encoding, target_encode_test


In [9]:
data = pd.read_csv(f"/Users/yuke/Documents/Codes/DSC/DSC_STUDY/FinDataChallenge/takazawa/data/train.csv", index_col=0)
testdata = pd.read_csv(f"/Users/yuke/Documents/Codes/DSC/DSC_STUDY/FinDataChallenge/takazawa/data/test.csv", index_col=0)

## Cleaning
`clean_data`でデータのクリーニングができます
- \$マークやカンマを除いて数値列に
- 日付のデータはdatetimeにし，"{column}\_year", "{column}\_month", "{column}\_day" "{column}_{daystamp}"(一番古い日から何日経ったか)の列が追加されています．
- カテゴリカルへの変換
- Franchise == 0 かどうかと，Franchise==1 かどうかの列を作る.

このデータは`cleaned_train.csv`, `cleaned_test.csv`に保存しています．

ここまでと，`State`, `BankState`, `Sector`のholdout target encoding（後述）をして，LightGBMの通常の設定でcv (with Macrof1 score) + 予測の閾値を適切に選べば，スコア`0.68`くらいまで出せるはずです．

In [10]:
# 元データ
data

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural
0,163,21,1.0,0,0,1,N,N,31-Jan-98,1,0,22-Sep-06,2006,PHOENIX,AZ,SD,"$80,000.00","$80,000.00","$68,000.00",0
1,84,6,1.0,4,0,0,0,N,31-Oct-93,1,62,30-Jun-92,1992,MCALESTER,OK,OK,"$287,000.00","$287,000.00","$229,600.00",0
2,242,45,1.0,4,90,0,N,N,31-Aug-01,1,42,18-Apr-01,2001,HAWTHORNE,NJ,NJ,"$31,983.00","$30,000.00","$15,000.00",1
3,237,4,1.0,0,0,0,N,N,31-Aug-07,1,33,6-Oct-03,2004,NASHVILLE,TN,SD,"$229,000.00","$229,000.00","$229,000.00",0
4,184,0,1.0,0,0,0,N,N,8-Jun-83,1,0,17-Dec-99,2000,POMONA,CA,CA,"$525,000.00","$525,000.00","$393,750.00",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42302,283,14,1.0,0,0,1,N,N,31-Jan-98,1,0,2-Mar-95,1995,PHILADELPHIA,PA,PA,"$80,000.00","$80,000.00","$68,000.00",0
42303,53,2,1.0,0,0,0,Y,N,3-Apr-91,1,42,6-Jun-07,2007,LOS ANGELES,CA,SD,"$5,000.00","$5,000.00","$4,250.00",1
42304,59,6,2.0,0,0,1,N,N,28-Feb-03,1,42,14-Mar-03,2003,COLUMBUS,OH,OH,"$60,000.00","$60,000.00","$51,000.00",0
42305,295,18,1.0,0,8,0,N,N,10-Dec-97,1,42,23-Aug-89,1989,CLOQUET,MN,MN,"$294,000.00","$294,000.00","$220,500.00",0


In [11]:
data = clean_data(data)
testdata = clean_data(testdata)

In [12]:
# cleaning後のタイプ
data.dtypes

Term                                  int64
NoEmp                                 int64
NewExist                           category
CreateJob                             int64
RetainedJob                           int64
FranchiseCode                      category
RevLineCr                          category
LowDoc                             category
DisbursementDate             datetime64[ns]
MIS_Status                            int64
Sector                             category
ApprovalDate                 datetime64[ns]
ApprovalFY                            int64
City                                 object
State                                object
BankState                            object
DisbursementGross                   float64
GrAppv                              float64
SBA_Appv                            float64
UrbanRural                         category
DisbursementDate_year               float64
DisbursementDate_month              float64
DisbursementDate_day            

In [13]:
data.to_csv("edited_data/cleaned_train.csv")
testdata.to_csv("edited_data/cleaned_test.csv")

## Basic count and rank encoding for Multicategory

- 使えるかどうかはわかっていません．
- `Sector`, `State`, `BankState`, `City`, `FranchiseCode` に対してやっています．
- count encoding した後の列は `{column}_count`, count rank encoding した後の列は `{column}_count_rank`に入っています．

ここまで整形したデータを，`basic_encoded_train.csv`, `basic_encoded_test.csv`に入れています．（test データの count encodingはtrainの情報を使ってもいいかもしれません．．．今はtestのものだけ使っています）

In [14]:
multi_cat_cols = ['Sector', 'State', 'BankState', 'City', 'FranchiseCode']
data = count_encoding(data, multi_cat_cols)
data = count_rank_encoding(data, multi_cat_cols)
testdata = count_encoding(testdata, multi_cat_cols)
testdata = count_rank_encoding(testdata, multi_cat_cols)

In [15]:
data[[item + "_count" for item in multi_cat_cols]]

,Sector_count,State_count,BankState_count,City_count,FranchiseCode_count
0,9798,768,2382,324,14033
1,1191,527,453,8,26392
2,7337,488,195,6,26392
3,5050,1147,2382,599,26392
4,9798,6893,6476,584,26392
...,...,...,...,...,...
42302,9798,2849,1307,612,14033
42303,7337,6893,2382,348,26392
42304,7337,1229,2785,124,14033
42305,7337,1004,824,2,26392


In [16]:
data[[item + "_rank" for item in multi_cat_cols]]

,Sector_rank,State_rank,BankState_rank,City_rank,FranchiseCode_rank
0,1.0,17.0,7.0,22.0,2.0
1,9.0,23.0,24.0,593.0,1.0
2,2.0,25.0,40.0,769.5,1.0
3,3.0,8.0,7.0,5.0,1.0
4,1.0,1.0,1.0,6.0,1.0
...,...,...,...,...,...
42302,1.0,4.0,9.0,4.0,2.0
42303,2.0,1.0,7.0,16.0,1.0
42304,2.0,6.0,4.0,70.5,2.0
42305,2.0,10.0,12.0,1626.5,1.0


In [17]:
data.to_csv("edited_data/basic_encoded_train.csv")
testdata.to_csv("edited_data/basic_encoded_test.csv")

## Holdout Target Encoding

カテゴリー数が多いカテゴリカルに対して，目的変数の値を使ってターゲットエンコーディングをします．（leakが起きている状況なので，overfitに注意）

`Holdout_target_encoding` と `target_encode_test` を次の例を参考に使ってください．

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import  KFold

In [22]:
X_train, X_test, y_train, y_test = train_test_split(data, data["MIS_Status"], test_size=0.2, random_state=42) # stratifyした方がいいかも

In [24]:
target_encode_cols = ['Sector', 'State', 'BankState']
kf = KFold(n_splits=3, shuffle=True, random_state=1000)
# for train
kf_iter_train = kf.split(X_train)

# create list of indices for training / test data to use for holdout target encoding
folds_train = []
for train_idx, test_idx in kf.split(X_train):
    folds_train.append((train_idx, test_idx))

# holdout target encoding with the folds
for col in target_encode_cols:
    X_train = Holdout_target_encoding(X_train, y_train, col, folds_train) 
    
# We can simply use training data to encode the test data
for col in target_encode_cols:
    X_test = target_encode_test(X_train, y_train, X_test, col)

In [25]:
target_encoded_cols = [item + "_target" for item in target_encode_cols]
X_train[target_encoded_cols]

,Sector_target,State_target,BankState_target
6863,0.837030,0.900193,0.873469
30454,0.882296,0.902727,0.920869
8111,0.912308,0.942085,0.916484
22811,0.914838,0.885714,0.895833
4428,0.837030,0.914692,0.928109
...,...,...,...
6265,0.900051,0.902727,0.876604
11284,0.912308,0.903271,0.905093
38158,0.943460,0.882924,0.882403
860,0.943460,0.882924,0.882403


## 簡単なモデルの例

### 使う変数

In [26]:
num_cols = ['NoEmp', 'CreateJob', 'RetainedJob', 'ApprovalFY', 'DisbursementGross', 'GrAppv', 'SBA_Appv']
retained_cat_cols = ['NewExist', 'RevLineCr', 'LowDoc', 'UrbanRural']
timestamp_cols = ['DisbursementDate_daystamp', 'ApprovalDate_daystamp']
franchise_cols = ['FranchiseCode1', 'FranchiseCode0']
target_encoded_cols = [item + "_target" for item in target_encode_cols]
all_cols = num_cols + retained_cat_cols + timestamp_cols + franchise_cols + target_encoded_cols

### Macro f1 metric を定義

In [27]:
from sklearn.metrics import  f1_score

In [33]:
def Macrof1(preds, eval_dataset):
    y_true = eval_dataset.get_label()
    y_pred = (preds>0.5).astype(int)
    score = f1_score(y_true, y_pred, average='macro')
    return 'Macrof1', score, True

### LightGBM

In [30]:
import lightgbm as lgb

In [34]:
params = {
    'objective': 'binary',
    'metric': 'custom',  # Use custom to use the custom metric for evaluation
    'verbose': 1,
    'early_stopping_rounds' : 100
}
dataset = lgb.Dataset(X_train[all_cols], label=y_train)

# Define CV parameters
cv_results = lgb.cv(
    params,
    dataset,
    num_boost_round=1000,
    nfold=5,
    feval=Macrof1,  # Custom evaluation function
    stratified=False,
    seed=42,
    return_cvbooster=True
)

[LightGBM] [Info] Number of positive: 24180, number of negative: 2896
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002316 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1979
[LightGBM] [Info] Number of data points in the train set: 27076, number of used features: 18
[LightGBM] [Info] Number of positive: 24128, number of negative: 2948
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002267 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1979
[LightGBM] [Info] Number of data points in the train set: 27076, number of used features: 18
[LightGBM] [Info] Number of positive: 24157, number of negative: 2919
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing wa

In [41]:
boosters = cv_results['cvbooster']

In [47]:
pred_average = np.mean(boosters.predict(X_test[all_cols]), axis=0)
test_prediction = (pred_average > np.quantile(pred_average, 0.1)).astype(int) # ここはもう少し良い選び方があるはずです．

In [48]:
f1_score(y_test, test_prediction, average='macro')

0.6703800305701028